In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

In [ ]:
train=pd.read_csv('Google_Stock_Price_Train.csv')

In [ ]:
training_set=train.iloc[:,1:2].values

In [ ]:
sc=MinMaxScaler()
training_set_scaled=sc.fit_transform(training_set)

In [ ]:
X_train=[]
y_train=[]
for i in range(60,len(training_set_scaled)):
    X_train.append(training_set_scaled[i-60:i,0])
    y_train.append(training_set_scaled[i,0])
X_train,y_train=np.array(X_train),np.array(y_train)

In [ ]:
X_train=np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))

In [ ]:
from keras.models import Sequential
from keras.layers import Dropout,LSTM,Dense

In [ ]:
regressor=Sequential()

In [ ]:
regressor.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1],1)))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units=50,return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units=50,return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units=50))
regressor.add(Dense(units=1))
regressor.compile(optimizer='adam',loss='mean_squared_error')
regressor.fit(X_train,y_train,epochs=100,batch_size=32)

In [ ]:
test=pd.read_csv('Google_Stock_Price_Test.csv')
stock_price=test.iloc[:,1:2].values

In [ ]:
combine=pd.concat((train['Open'],test['Open']),axis=0)
inputs=combine[len(combine)-len(stock_price)-60:].values
inputs=inputs.reshape(-1,1)
inputs=sc.transform(inputs)

In [ ]:
X_test=[]
for i in range(60,80):
    X_test.append(inputs[i-60:i,0])
X_test=np.array(X_test)
X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))

In [ ]:
predicted=regressor.predict(X_test)
predicted=sc.inverse_transform(predicted)

In [ ]:
plt.plot(stock_price,color='red',label='real stock price')
plt.plot(predicted,color='blue',label='predicted stock price')
plt.title('Google stock price prediction')
plt.xlabel('Time')
plt.ylabel('Google stock price')
plt.legend()
plt.show()